In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
trans = pd.Series([pd.to_datetime('2019-0{}-{}'.format(a,x)) 
                   for a,x in zip(np.random.choice([8,9],size=100),np.random.randint(1,31,size=100))])

In [86]:
df = trans.to_frame('date')
df['value'] = 1000*np.random.random(100)
df['id'] = np.random.randint(11,size=100)
df['transac_desc'] = np.random.choice(['A','B','C','D'],size=100)

This is the synthetic data I've generated:

In [87]:
df.head()

,date,value,id,transac_desc
0,2019-08-18,99.996527,9,A
1,2019-08-15,164.329553,1,D
2,2019-08-15,698.991704,4,C
3,2019-09-07,150.666775,4,A
4,2019-08-19,424.157114,10,C


Here is a very inefficient way of getting the intervals between dates:

In [88]:
df['freq_transac']=df.groupby(['id','transac_desc'])['date'].transform(lambda x : (x - min(x)).dt.days)

In [89]:
def teste(x):
    x = x.sort_values()
    if x.size > 1:
        x = x - x.shift(1)
    return x

In [90]:
df['interval_transac']=df.groupby(['id','transac_desc'])['freq_transac'].transform(teste)

In [91]:
df.sort_values(by=['id','transac_desc','date']).head(20)

,date,value,id,transac_desc,freq_transac,interval_transac
91,2019-09-19,949.678114,0,A,0,NaN
97,2019-09-24,179.361012,0,A,5,5.0
98,2019-08-02,581.116097,0,B,0,14.0
40,2019-08-08,827.573425,0,B,6,6.0
78,2019-08-29,539.465297,0,B,27,6.0
20,2019-09-04,854.591325,0,B,33,NaN
66,2019-09-18,379.526835,0,B,47,21.0
12,2019-09-04,100.996300,0,D,0,0.0
72,2019-08-01,617.447275,1,A,0,11.0
67,2019-08-18,472.789655,1,A,17,17.0


Much simpler, efficient and faster way than doing what I did previously (it has the bonus of placing the intervals in the correct rows):

In [92]:
df['date_diff'] = df.sort_values(by='date').groupby(['id','transac_desc'])['date'].diff().dt.days

In [93]:
df.sort_values(by=['id','transac_desc','date']).head(20)

,date,value,id,transac_desc,freq_transac,interval_transac,date_diff
91,2019-09-19,949.678114,0,A,0,NaN,NaN
97,2019-09-24,179.361012,0,A,5,5.0,5.0
98,2019-08-02,581.116097,0,B,0,14.0,NaN
40,2019-08-08,827.573425,0,B,6,6.0,6.0
78,2019-08-29,539.465297,0,B,27,6.0,21.0
20,2019-09-04,854.591325,0,B,33,NaN,6.0
66,2019-09-18,379.526835,0,B,47,21.0,14.0
12,2019-09-04,100.996300,0,D,0,0.0,NaN
72,2019-08-01,617.447275,1,A,0,11.0,NaN
67,2019-08-18,472.789655,1,A,17,17.0,17.0


In [94]:
df['day'] = df['date'].dt.day

In [95]:
def count_avg(x):
    return x.size/3

In [96]:
final= df.groupby(['id','transac_desc']).agg({'interval_transac':['mean','std'],'day':['min','mean','max','median','std',count_avg,lambda x: max(x)-min(x)],'value':['mean','std']})

In [97]:
final.columns

MultiIndex(levels=[['interval_transac', 'day', 'value'], ['<lambda>', 'count_avg', 'max', 'mean', 'median', 'min', 'std']],
           codes=[[0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2], [3, 6, 5, 3, 2, 4, 6, 1, 0, 3, 6]])

In [98]:
final.columns.get_level_values(0)

Index(['interval_transac', 'interval_transac', 'day', 'day', 'day', 'day',
       'day', 'day', 'day', 'value', 'value'],
      dtype='object')

In [99]:
final.columns = ['_'.join(col).strip() for col in final.columns.values]

In [100]:
final.query('interval_transac_std<3')

,,interval_transac_mean,interval_transac_std,day_min,day_mean,day_max,day_median,day_std,day_count_avg,day_<lambda>,value_mean,value_std
id,transac_desc,,,,,,,,,,,
2,D,4.000000,2.645751,12,16.750000,24,15.5,5.500000,1.333333,12,293.717424,305.756499
4,C,20.500000,0.707107,4,14.666667,25,15.0,10.503968,1.000000,21,426.914926,238.122071
7,B,9.333333,1.527525,1,12.250000,21,13.5,9.069179,1.333333,20,408.807671,425.080809


In [101]:
np.unique(final.index.get_level_values(0)).shape[0]

11